In [1]:
import os
os.chdir("/content/drive/My Drive/Humana Google Colab")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv("2020_Competition_Training (1).csv")
df["train"] = 1
df_hold = pd.read_csv("2020_Competition_Holdout .csv")
df_hold["train"] = 0
df_hold["transportation_issues"] = None
df_all = pd.concat([df, df_hold])
df_all.reset_index(inplace=True)
df_all.shape #87253 rows with 828 features

(87253, 828)

In [3]:
df_feature_selection = pd.read_csv("features kept2.csv")
features_selection = df_feature_selection["0"].tolist() + ["train", "transportation_issues"]
df = df[features_selection] 

In [4]:
df.shape #We have 61columns, 1 for train identification, 1 for target, and 59 for features.

(69572, 61)

### Missing Values

In [5]:
df_columns = df.columns.tolist()
df_columns.remove("transportation_issues")

#Columns with missing value
columns_with_missing = []
for column in df_columns:
    if df[column].isnull().sum()>0:
        columns_with_missing.append({"column":column, "missing":df_all[column].isnull().sum()})

df_missing_distribution = pd.DataFrame.from_dict(columns_with_missing)
df_missing_distribution.sort_values("missing",ascending=False).reset_index(inplace=True)
from sklearn.impute import SimpleImputer
# rank 1 blood fat screen or control
# rank 2 data from cons about demos
# rank 3 cms data insurance factor data.
imputer_frequent = SimpleImputer(strategy="most_frequent")
for column in df_missing_distribution["column"]:
    try:
        if df[column].nunique()>=30:
            df[column].fillna(df[column].mean(),inplace=True)
        else:
            df[column].fillna(df[column].value_counts().index[0],inplace=True)
    except:
        print(column)

## Variable Types and Manipulation

In [6]:
column_types = pd.DataFrame(df.dtypes).reset_index()
object_columns = column_types[column_types[0]=="object"]["index"].tolist()
float_columns = column_types[column_types[0]=="float64"]["index"].tolist()
int_columns = column_types[column_types[0]=="int64"]["index"].tolist()
print("object:{},  float:{},  int:{}".format(len(object_columns),len(float_columns),len(int_columns)))

object:1,  float:44,  int:16


In [7]:
df.drop("state_cd",axis=1, inplace=True)

## Split the data

In [8]:
df.shape

(69572, 60)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df,\
                                                    df.transportation_issues, test_size=0.3,
                                                    random_state=42)

In [10]:
df = pd.get_dummies(df)
print(df.dtypes)

betos_o1g_pmpm_ct                        float64
bh_dema_ind                                int64
cci_score                                float64
ccsp_014_ind                               int64
ccsp_020_ind                               int64
ccsp_236_ind                               int64
ccsp_239_ind                               int64
ccsp_242_ind                               int64
cms_disabled_ind                           int64
cms_dual_eligible_ind                      int64
cms_hospice_ind                            int64
cms_low_income_ind                         int64
cms_partd_ra_factor_amt                  float64
cmsd2_skn_radiation_ind                    int64
cons_hcaccprf_h                          float64
cons_veteran_y                           float64
credit_bal_1stmtg_30to59dpd              float64
credit_bal_1stmtg_60to89dpd              float64
credit_bal_1stmtg_collections            float64
credit_bal_1stmtgcredit_60dpd            float64
credit_bal_autofinan

## Upsampling

In [11]:
from sklearn.utils import resample
X = X_train.copy()
# separate minority and majority classes
wo_tran = X[X["transportation_issues"]==0]
tran = X[X["transportation_issues"]==1]

# upsample minority
tran_upsampled = resample(tran,
                          replace=True, # sample with replacement
                          n_samples=len(wo_tran), # match number in majority class
                          random_state=1) # reproducible results

upsampled = pd.concat([wo_tran,tran_upsampled])

# check new class counts
print(upsampled["transportation_issues"].value_counts())

X_train = upsampled.drop("transportation_issues",axis=1)
y_train = upsampled["transportation_issues"].astype(int)
X_test = X_test.drop("transportation_issues", axis=1)
y_test = y_test.astype(int)

1    41536
0    41536
Name: transportation_issues, dtype: int64


## Transformation & Scaling

In [12]:
#Not sure about this step. But it is very important. Let's leave it for here.
# from sklearn.preprocessing import PowerTransformer
# pt = PowerTransformer().fit(X_train)
# X_train = pt.transform(X_train)

# pt = PowerTransformer().fit(X_test)
# X_test = pt.transform(X_test)

In [13]:
#Robust Scaler
from sklearn.preprocessing import RobustScaler

rs = RobustScaler().fit(X_train)
X_train = rs.transform(X_train)

rs = RobustScaler().fit(X_test)
X_test = rs.transform(X_test)

In [14]:
# X_train[np.isnan(X_train)] = 0
# X_test[np.isnan(X_test)] = 0

## Basic Logistic Model - No Improvement Found Yet.

In [15]:
#Lets do a simple logistic regression here first 
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver = "lbfgs",random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.6618992657832807

In [16]:
y_train_pred = clf.predict(X_train)
(y_train_pred == y_train).mean()
roc_auc_score(y_train_pred, y_train)
# it is underfitting

0.6619564759755088

In [17]:
#Accuracy
(y_test==y_pred).mean() #Underfitting

0.673486009965504

In [18]:
from xgboost import XGBClassifier

model = XGBClassifier(
        n_estimators=5000,
        max_depth=3, 
        learning_rate=0.01)

xgboost_model = model.fit(X_train, y_train)
y_pred = xgboost_model.predict(X_test)
print(roc_auc_score(y_test, y_pred))
print((y_test==y_pred).mean())

0.6605143097064824
0.6900632426216942
